# YOLOv5 ball detection
### References

* YOLOv5 repository - https://github.com/ultralytics/yolov5
* https://www.kaggle.com/code/eneszvo/yolov5-helmet-detection-train-and-inference

In [3]:
!nvidia-smi

Thu Sep  8 07:15:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   48C    P8    37W / 350W |    610MiB / 24576MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.2+cu113 (NVIDIA GeForce RTX 3090)


In [5]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import subprocess

import glob

In [6]:
class CFG:
    TRAIN_DIR = "/workdir/work/input/train"

    YOLO_PATH = "/workdir/work/yolov5"

    CROPED_IMG_DIR = "/workdir/work/output/croped_image"
    DET_TRAIN_MOVIES_DIR = "/workdir/work/yolov5/TRAIN_MOVIES"

    IMG_SIZE = 1280
    IMG_HEIGHT = 768
    IMG_WIDTH = 1280
    calspeed_frame_num = 25 #移動距離を計算するフレーム数
 
    

In [7]:
!ls

Dockerfile  cuda-keyring_1.0-1_all.deb	requirements.txt  work
README.md   docker-compose.yaml		run.sh


In [8]:
%cd {CFG.YOLO_PATH}

/workdir/work/yolov5


In [9]:
!ls

08fd33_0.mp4		  classify	       requirements.txt
BALL_DET_SAMPLE_SAVECONF  data		       setup.cfg
CONTRIBUTING.md		  detect.py	       train.py
DFL			  detect_edit.py       tutorial.ipynb
LICENSE			  export.py	       utils
README.md		  hubconf.py	       val.py
TRAIN_MOVIES		  models	       yolov5l6_trained_600images.pt
__pycache__		  output_ball_det.mp4


# ボールの移動距離が長いところだけ切り取ってクロップしたい

In [10]:
def yolobbox_to_pixel(bbox_, WIDTH=CFG.IMG_WIDTH, HEIHGT=CFG.IMG_HEIGHT):
    x_min = int( (bbox[0] - bbox[2])*WIDTH )
    y_min = int( (bbox[1] - bbox[3])*HEIHGT )
    x_max = int( (bbox[0] + bbox[2])*WIDTH )
    y_max = int( (bbox[1] + bbox[3])*HEIHGT )
    return [x_min, y_min, x_max, y_max]

In [11]:
def yolobbox_to_croparea(bbox_, WIDTH=CFG.IMG_WIDTH, HEIHGT=CFG.IMG_HEIGHT):
    """
    検出したボールを中心として、CROP_AREA_X*2, CROP_AREA_Y*2の範囲をクロップする予定
    そのための座標情報を入力する。
    """
    x_mid = int( bbox_[0]*WIDTH )
    y_mid = int( bbox_[1]*HEIHGT )
    
    CROP_AREA_X = WIDTH // 6
    CROP_AREA_Y = HEIHGT // 6  
    x_min = x_mid - CROP_AREA_X
    x_max = x_mid + CROP_AREA_X
    if x_min < 0:
        x_min += abs(x_min)
        x_max += abs(x_min)
    elif x_max > WIDTH:
        x_min -= (x_max - WIDTH)
        x_max -= (x_max - WIDTH)

    y_min = y_mid - CROP_AREA_Y
    y_max = y_mid + CROP_AREA_Y
    if y_min < 0:
        y_min += abs(y_min)
        y_max += abs(y_min)
    elif y_max > HEIHGT:
        y_min -= (y_max - HEIHGT)
        y_max -= (y_max - HEIHGT)

    return [x_min, y_min, x_max, y_max]

In [12]:
def calculate_distance(bbox1_, bbox2_):# もしかしたらあとでなんか変えるかも？？
    return (bbox1_[0] - bbox2_[0])**2 + (bbox1_[1] - bbox2_[1])**2

In [13]:
def calculate_speed(bbox1_, bbox2_):# とりあえずフレーム間の距離をspeedとして扱うことにする。
    height_rate = 4 # 画面内での横の移動より縦の移動のほうが実際の移動距離は長い。適当な補正をかける。
    return (bbox1_[0] - bbox2_[0])**2 + (height_rate * (bbox1_[1] - bbox2_[1])**2)

In [14]:
def pickup_play_scene(bbox_frames_list_, ball_bbox_, image_, speed_thr=0.0010):
    """
    input 
        bbox_frames_list_   : 現時刻のフレームまでの{calspeed_frame_num}フレーム分のボールのbboxのリスト
        ball_bbox           : 現時刻のフレームで検出したボールのbbox
        image_              : 現時刻のフレームの画像
    
    output
        bbox_frames_list_   : 現時刻のフレームのボールbbox追加後のbboxリスト
        is_move_detected    : bbox_frames_list内のbbox位置と現時刻のbbox位置との移動距離が閾値以上でTrue
    """
    
    if len(bbox_frames_list_) > calspeed_frame_num:
        bbox_frames_list_ = bbox_frames_list_[1:calspeed_frame_num]
    speed = 0
    for bbox_ in bbox_frames_list_:
        # calculate 10frames speed of ball
        speed = max(calculate_speed(bbox_, ball_bbox_), speed)
    bbox_frames_list_.append(ball_bbox_)

    is_move_detected = speed > speed_thr
    
    return bbox_frames_list_, is_move_detected

In [15]:
def CropImage_and_MakeFrameBboxList(movie_id_, frame_id_, fps_, bbox_frames_list_, framebbox_list_, saved_frame_list_, rewind_time=1.0):
    rewind_idx = int(rewind_time*fps_)
    
    if len(bbox_frames_list_) - rewind_idx <= 1:
        rewind_idx = len(bbox_frames_list_)
    list_pick_idx = -rewind_idx

    for cropping_frame_idx in range(abs(rewind_idx)):
        #そのときのフレームidに巻き戻し分を引いて、rangeでずらしたのがcropしたいframeのid
        cropping_frame = cropping_frame_idx - rewind_idx + frame_id_
        
        cropping_framebbox = [cropping_frame]
        cropping_framebbox.extend(bbox_frames_list_[list_pick_idx])
        if cropping_frame in saved_frame_list_:# すでにcropして保存済みであればcontinue
            list_pick_idx += 1
            continue

        else:# まだ保存してない場合は保存処理
            framebbox_list_.append(cropping_framebbox)
            crop_area = yolobbox_to_croparea(bbox_frames_list_[list_pick_idx]) # クロップ位置の計算
            cap.set(cv2.CAP_PROP_POS_FRAMES, cropping_frame) # 動画のフレーム位置を指定フレームまで巻き戻す
            ret, frame_image = cap.read()# 指定フレームの画像読み込み
            if not ret:
                continue
            else:
                # 指定位置をクロップ
                frame_image = cv2.resize(frame_image, dsize=(CFG.IMG_WIDTH, CFG.IMG_HEIGHT))
                croped_image = frame_image[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2],  :]
                # 保存用ディレクトリを作成
                image_save_dir = f"{CFG.CROPED_IMG_DIR}/{movie_id_}"
                os.makedirs(image_save_dir, exist_ok=True)
                # Crop画像の保存
                croped_image_filename = f"{image_save_dir}/croped_{movie_id_}_{cropping_frame}.jpg"
                cv2.imwrite(croped_image_filename, croped_image)
            saved_frame_list_.append(cropping_frame)
            list_pick_idx += 1
        
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id_) # 現在のフレーム位置に戻す
    
    return framebbox_list_, saved_frame_list_

In [16]:
train_movies = glob.glob(f"{CFG.TRAIN_DIR}/*.mp4")

for cropping_movie in train_movies:
    # DEBUG用
    # if cropping_movie != "/workdir/work/input/train/3c993bd2_0.mp4":
    #     break

    # movie_idの設定
    movie_id = movie_id = cropping_movie[26:-4]
    # YOLOv5の検出結果の読み込み
    MOVIE_DET_PATH = f"{CFG.DET_TRAIN_MOVIES_DIR}/{movie_id}/labels"
    bbox_files = glob.glob(f"{MOVIE_DET_PATH}/*.txt")

    print(f"Movie={movie_id}, Detected object num = {len(bbox_files)}.")

    # videoの読み込み
    cap = cv2.VideoCapture(f"{CFG.TRAIN_DIR}/{movie_id}.mp4")
    frame_num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # 初期化
    bbox_frames_list = [] # 複数フレームのbboxを格納したリスト
    framebbox_list = [] # [frame_id, bbox]のリスト
    saved_frame_list = [] # 保存したframe_idのリスト
    calspeed_frame_num = CFG.calspeed_frame_num #ここで指定したフレーム分だけ移動距離を計算する(最大値を取る)
    
    print(f"Cropping image if ball is moved. Movie_id={movie_id}, frames={frame_num}, fps={fps}")

    for frame_idx in tqdm(range(frame_num)):
        frame_id = frame_idx+1
        # Read frame image (cap.readはメソッドが実行されるたびに1フレーム分進める)
        ret, image = cap.read()
        if not ret:
            print(f"frame:{frame_id} doesn's exist.")
            continue

        # DEBUG用
        # if cap.get(cv2.CAP_PROP_POS_FRAMES) < 6000:
        #     continue
        # if cap.get(cv2.CAP_PROP_POS_FRAMES) > 6100:
        #     break

        det_file_name = f"{MOVIE_DET_PATH}/{movie_id}_{frame_id}.txt"
        if det_file_name in bbox_files:
            # select highest conf bbox in txt-file
            with open(det_file_name, "rb") as f:
                det_file_data = f
                bbox_txt = [s.strip() for s in det_file_data.readlines()]

            bbox_tmp = []
            ball_bbox = []
            if len(bbox_txt) > 1:
                # 最初の検出ではconfが最も高いものを検出結果として選ぶ
                if len(bbox_tmp) == 0:
                    best_conf = .0
                    for bbox in bbox_txt:
                        conf_and_bbox_list = str(bbox).strip("'").split(" ")
                        conf_and_bbox = np.float_(conf_and_bbox_list[1:])
                        if best_conf < conf_and_bbox[4]:
                            best_conf = conf_and_bbox[4]
                            ball_bbox = conf_and_bbox[0:4]
                # 2回目以降は検出値とユークリッド距離が近いものを検出とする
                else:
                    nearest_distance = 1e10
                    for bbox in bbox_txt:
                        conf_and_bbox_list = str(bbox).strip("'").split(" ")
                        conf_and_bbox = np.float_(conf_and_bbox_list[1:])
                        distance = calculate_distance(conf_and_bbox, bbox_tmp)
                        if nearest_distance > distance:
                            nearest_distance = distance
                            ball_bbox = conf_and_bbox[0:4]

            else:
                conf_and_bbox_list = str(bbox_txt).strip("'").split(" ")
                ball_bbox = np.float_(conf_and_bbox_list[1:5])
        
            # 前回の検出位置を格納しておく
            bbox_tmp = ball_bbox
            # detectしてないときは前回値をそのまま使う
            bbox_frames_list, is_move_detected = pickup_play_scene(bbox_frames_list, ball_bbox, image)

            # ボールの移動判定が入ったら、0.5秒(関数内で指定した秒数)分の戻ったフレームから画像をcropして保存する
            if is_move_detected:
                framebbox_list, saved_frame_list = CropImage_and_MakeFrameBboxList(movie_id, frame_id, fps, bbox_frames_list, framebbox_list, saved_frame_list)

    # cropした箇所のframe数とbbox位置をcsvで保存する
    croped_frame_df = pd.DataFrame(framebbox_list, columns=["frame_id", "bbox_xmid", "bbox_ymid", "bbox_height", "bbox_width"])
    croped_csvfile_name = f"{CFG.CROPED_IMG_DIR}/{movie_id}/croped_frame.csv"
    croped_frame_df.to_csv(croped_csvfile_name, index=False)

Movie=3c993bd2_0, Detected object num = 47214.
Cropping image if ball is moved. Movie_id=3c993bd2_0, frames=89750, fps=25


  7%|█████████▋                                                                                                                             | 6424/89750 [02:29<4:29:22,  5.16it/s]